In [1]:
import pandas as pd
from konlpy.tag import Mecab
import pickle
# torch import
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torchtext
from collections import Counter
from torchtext import transforms as T

In [3]:
train = pd.read_csv('/root/share/data/train.csv')
test = pd.read_csv('/root/share/data/test.csv')
print(len(train))
print(len(test))


40000
5000


In [4]:
train = train[train['data'].notnull()]

In [5]:
train['data'][6].replace('\\n','')

"'초중고학교 페미니즘교육의무화'중복동의인원 무효처리해 주세요!!. 국민들의 요구를 묵살해도 되나요? 명백한 증거가 있고 부정한 방법을 동원해 목표달성인원을 충족한 해당 청원에 대하여 청와대는 왜 아무런 조치를 취하지 않나요?? 중복 동의된 인원은 무효처리 후 인원 수를 다시 계상해야 정상적이지 않나요?? 청와대가 언제부터 부정한 청원에 대하여 방관하고 용인하였나요??잘못된 부분은 시정하고 바로잡아야 하지 않나요??? 일국의 청와대씩이나 돼 갖고...21만여 표 중에서 중복처리된 부분은 무효처리한 후 다시 숫자를 집계하여 주세요...하물며 정부답변 청원이쟎아요...청와대가 이래서야 되겠습니까??인터넷은 전 세계인이 들락날락하는 공간이에요..어휴, 창피해!!"

In [6]:
def tokenizer(sentence, tag):
    return tag.morphs(sentence)
mecab = Mecab()
train['data'] = train['data'].apply(lambda x: x.replace('\\n', ''))
train['data'] = train['data'].apply(lambda x: x.replace('.', ' '))
train['morph'] = train['data'].apply(tokenizer, tag=mecab)

In [7]:
counter = Counter()
for sentence in train['morph']:
    counter.update(sentence)

In [8]:
from torchtext.vocab import vocab
from collections import Counter
import torchtext

In [9]:
vocab_f = vocab(counter, min_freq=5, specials=['<unk>','<pad>'])

In [10]:
vocab_f.set_default_index(vocab_f['<unk>'])

In [15]:
output = T.VocabTransform(vocab_f)(train['morph'].tolist())

/opt/conda/lib/python3.7/site-packages/torch/_jit_internal.py:1138: UserWarning: The inner type of a container is lost when calling torch.jit.isinstance in eager mode. For example, List[int] would become list and therefore falsely return True for List[float] or List[str].
  warnings.warn("The inner type of a container is lost when "


In [17]:
output = T.ToTensor(padding_value=vocab_f['<pad>'])(output)

In [16]:
output = T.Truncate(max_seq_len=300)(output)

In [18]:
output.size()

torch.Size([39992, 300])

In [21]:
text_transform = T.Sequential(
    T.VocabTransform(vocab_f),
    T.Truncate(max_seq_len=300),
    T.ToTensor(padding_value=vocab_f['<pad>'])
)

preprocess_output = text_transform(train['morph'].tolist())

/opt/conda/lib/python3.7/site-packages/torch/_jit_internal.py:1138: UserWarning: The inner type of a container is lost when calling torch.jit.isinstance in eager mode. For example, List[int] would become list and therefore falsely return True for List[float] or List[str].
  warnings.warn("The inner type of a container is lost when "


In [23]:
preprocess_output.size()

torch.Size([39992, 300])

In [30]:
label = train.category.tolist()

In [ ]:
vocab_f

In [32]:
import pickle

with open('/root/share/data/vocab.pkl','wb') as f:
    pickle.dump(vocab_f, f)

with open('/root/share/data/train_data.pkl','wb') as f:
    pickle.dump(preprocess_output, f)

with open('/root/share/data/train_label.pkl', 'wb') as f:
    pickle.dump(label, f)